This notebook was used for development only, all the functional logic is already embbeded in dataset_loading notebook

# importing libraries

In [1]:

import pandas as pd
from tqdm import tqdm
import re
import emoji
import unicodedata as ud


tqdm.pandas()


# loading data and intial inspection

In [2]:
df=pd.read_csv("../data/interim/out.csv",sep="|")

In [3]:
df["tweet_len"]=df["tweet"].apply(lambda x: len(x))

In [6]:
def filter_text(text:str):
    flags={'U+1F1E6',
            'U+1F1E7',
            'U+1F1E9',
            'U+1F1EA',
            'U+1F1EC',
            'U+1F1ED',
            'U+1F1EE',
            'U+1F1EF',
            'U+1F1F0',
            'U+1F1F1',
            'U+1F1F2',
            'U+1F1F3',
            'U+1F1F4',
            'U+1F1F5',
            'U+1F1F6',
            'U+1F1F7',
            'U+1F1F8',
            'U+1F1F9',
            'U+1F1FC',
            'U+1F1FE',
            'U+1F1FF'}

    # remove emojis except for flags
    text  =''.join(char for char in text if (char in flags or not emoji.is_emoji(char)))
    # remove mentions
    text = re.sub("@[A-Za-z0-9_]+", "", text)
    # remove links
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    # remove hashes and keep the words
    text = text.replace("#", " ").replace("_", " ")
    # remove all english characters
    text = re.sub('[a-zA-Z0-9]+', '', text)
    # replace  multiple spaces with one space
    text = re.sub(' +', ' ',text)
    # remove all punctionations and all digits (arabic and english)
    text  =''.join(char for char in text if not (
                    ud.category(char).startswith('P') or 
                    ud.category(char).startswith('Nd')))
    return text
                   
   

In [5]:
df["clean"]=df["tweet"].progress_apply(filter_text)

100%|██████████| 458183/458183 [03:32<00:00, 2153.01it/s]


In [7]:
df

,id,tweet,label,tweet_len,clean
0,1175358310087892992,لكن بالنهاية ينتفض يغير,IQ,27,لكن بالنهاية ينتفض يغير
1,1175416117793349632,يعني هذا محسوب على البشر حيونه ووحشيه وتطلب...,IQ,98,يعني هذا محسوب على البشر حيونه ووحشيه وتطلبون...
2,1175450108898565888,مبين من كلامه خليجي,IQ,20,مبين من كلامه خليجي
3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ,26,يسلملي مرورك وروحك الحلوه
4,1175496913145217024,وين هل الغيبه اخ محمد,IQ,23,وين هل الغيبه اخ محمد
...,...,...,...,...,...
458178,1021088486915559424,مرو خذوني وياكم بالاحمر,BH,25,مرو خذوني وياكم بالاحمر
458179,1024943651569446784,هاي لو كنت حابه تاكلي شو طلبك,BH,30,هاي لو كنت حابه تاكلي شو طلبك
458180,1018588912648904832,الحين نسوي ربيان مشوي حياكم,BH,28,الحين نسوي ربيان مشوي حياكم
458181,1024945458576273408,الله يغفر لكم\nمساء الوررررد,BH,28,الله يغفر لكم\nمساء الوررررد


In [8]:

def remove_emoji(text):
    
    text  =''.join(char for char in text if  any ([not emoji.is_emoji(char),char in flags]))
                   
    return text


In [9]:
def remove_punct(text):
    
    text  =''.join(char for char in text if not (
        ud.category(char).startswith('P') or 
        ud.category(char).startswith('Nd')))
                   
    text = re.sub('[a-zA-Z0-9]+', '', text)
    return text


In [10]:

def remove_links(text):
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    return text


In [47]:
df["tweet_clean"] = df["tweet"].progress_apply(remove_emoji)

df["tweet_clean"] = df["tweet_clean"].progress_apply(lambda tweet: re.sub("@[A-Za-z0-9_]+", "", tweet))  # remove mentions

df["tweet_clean"] = df["tweet_clean"].progress_apply(lambda tweet: re.sub(' +', ' ',tweet))


df["tweet_clean"] = df["tweet_clean"].progress_apply(remove_links)


df["tweet_clean"] = df["tweet_clean"].progress_apply(lambda tweet: tweet.replace("#", " ").replace("_", " "))  # split hashtags

df["tweet_clean"] = df["tweet_clean"].progress_apply(remove_punct)

done
done
done
done
done


In [53]:
text="\n".join(df["tweet"])
with open("full tweets.txt","w") as f:
    f.write(text)

In [54]:
text="\n".join(df["tweet_clean"])

with open("full tweets clean.txt","w") as f:
    f.write(text)